In [168]:
from bs4 import BeautifulSoup
'''Доступ к Uniprot'''
from Bio import ExPASy
from Bio import SwissProt
import pandas as pd

columns = []
d = []
with open('With_Cys/pfam_1.txt', 'r') as f:
    for line in f:
        if line.startswith('<'):
            columns = line.split('> <')
        else:
            d.append(line.split())
columns[0] = columns[0].strip('<')
columns[-1] = columns[-1].strip('>\n')

data = pd.DataFrame(d, columns=columns)

with open("With_Cys/Cys_sequences_1.xml","r") as f:
    contents = f.read()
soup = BeautifulSoup(contents,'xml')

sequences = soup.sequences.findAll('sequence')

In [169]:
seqs = {}
for seq in sequences:
    for seg in seq.find_all('seg'):
        if bool(seg.find(idx = '0')) == True and bool(seg.find(idx = '1')) == True:
            hits = seg.find_all('hit')
            if float(hits[0]['pvalue']) <= 1e-15 and float(hits[1]['pvalue']) <= 1e-15:
                seqs.setdefault(seq['name'], []).append([int(hits[0]['pos']), int(hits[1]['pos'])])

In [170]:
with open('With_Cys/final_proteins_1.fasta', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line.lstrip('>')
        else:
            fasta.setdefault(name, []).append(line)

In [171]:
new_seq = {}
for key in seqs.keys():
    if key in fasta:
        new_seq[key] = seqs[key]
seqs = new_seq

In [172]:
data = data[data['seq id'].isin(seqs.keys())].reset_index(drop = True)

In [173]:
data.shape

(17300, 15)

In [174]:
def IfLOV(row):
    b = False
    if row['clan'] == 'CL0183':
        for hit in seqs[row['seq id']]:
            if (int(row[3]) - hit[0]) <=5 and (hit[1] + 43 - int(row[4])) <= 1:
                return 'LOV'
    return row['hmm name']
        

In [175]:
data['hmm name'] = data.apply(IfLOV, axis = 1)

In [177]:
from tqdm import tqdm

for name in tqdm(seqs.keys()):
    indexes = data[(data['seq id'] == name) & (data['hmm name'] == 'LOV')].index.values
    for i in indexes[:-1]:
        if (data.loc[i+1, 'hmm name'] == 'LOV' and 
            int(data.loc[i+1, 'envelope start']) - int(data.loc[i, 'envelope end']) <= 125):
            data.loc[i, 'envelope end'] = data.loc[i+1, 'envelope end']
            data.loc[i, 'hmm name'], data.loc[i+1, 'hmm name'] = 'TLOV', '_LOV'

100%|██████████| 5454/5454 [01:12<00:00, 74.86it/s] 


In [181]:
data.groupby(by = 'hmm name').groups

{'ADH_zinc_N': Int64Index([8171, 8220, 11137], dtype='int64'),
 'Aa_trans': Int64Index([1970], dtype='int64'),
 'BAT': Int64Index([2559, 7107, 7705, 10670, 11640, 16484, 16577], dtype='int64'),
 'CBS': Int64Index([409, 410, 7762, 7763, 8367, 8368, 13401], dtype='int64'),
 'CheB_methylest': Int64Index([3851, 9784, 14668], dtype='int64'),
 'CheR': Int64Index([3853, 9786, 14670], dtype='int64'),
 'CheR_N': Int64Index([3852, 9785, 14669], dtype='int64'),
 'Cupin_2': Int64Index([4919, 10855], dtype='int64'),
 'DUF1295': Int64Index([2379], dtype='int64'),
 'DUF3700': Int64Index([14403], dtype='int64'),
 'EAL': Int64Index([    9,    18,    35,   115,   277,   407,   434,  1117,  1199,
              1223,
             ...
             15921, 15939, 16233, 16234, 16418, 16623, 16642, 16667, 17070,
             17104],
            dtype='int64', length=136),
 'F-box': Int64Index([   25,    95,   139,   221,   237,   257,   283,   298,   360,
               425,
             ...
             1697